# Aula 07

Na aula passada começamos a trabalhar com o dataset do Titanic, fizemos a etapa de processamento. E a ideia é ir guardando o código que foi apresentando os resultados esperados.

In [1]:
import pandas as pd
import requests
from io import StringIO
import re

# Criação do dataFrame dos alunos
# ID do arquivo no Google Drive
file_id = '1S5Nl793vcL5ZPTGjzKaIEbwbLaDplvIP'

# URL modificada para forçar o download do arquivo
url = f"https://drive.google.com/uc?id={file_id}"

# Tentando obter o arquivo com requests
try:
    response = requests.get(url)
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    data = pd.read_csv(csv_raw)
except requests.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")


#Transformando o PassengerId no índice
data = data.set_index("PassengerId")

#Criando uma função que encontra os pronomes de tratamentos dos passageiros
def extract_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

#Aplicando a função na coluna de nomes, isto é, criando uma coluna só com os pronomes de tratamentos
data['Title'] = data['Name'].apply(extract_title)

#Substituindo os valores ausentes das idades pela mediana agrupada por sexo e classe de passageiro
data['Age'] = data.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

#Substituindo os valores ausentes do porto de embarque pelo valor que mais apareceu
data['Embarked'] = data['Embarked'].fillna('S')

#Substituindo os valores ausentes da cabine com base na Classe e no mapa do navio
for num in [1, 2, 3]:
    if num == 1:
        data.loc[data['Pclass'] == 1, 'Cabin'] = data.loc[data['Pclass'] == 1, 'Cabin'].fillna('ABC')
    elif num == 2:
        data.loc[data['Pclass'] == 2, 'Cabin'] = data.loc[data['Pclass'] == 2, 'Cabin'].fillna('DE')
    elif num == 3:
        data.loc[data['Pclass'] == 3, 'Cabin'] = data.loc[data['Pclass'] == 3, 'Cabin'].fillna('FG')


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     891 non-null    object 
 10  Embarked  891 non-null    object 
 11  Title     891 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 90.5+ KB


In [5]:
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,FG,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,FG,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,FG,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,DE,S,Rev
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.5,1,2,W./C. 6607,23.4500,FG,S,Miss


## Medidas de Tendência Central

### Média

A <u>média aritmética</u>, ou simplesmente média, de um conjunto de valores é a medida de centro encontrada somando todos os valores do conjunto e divindo pelo número de valores. Assim, se $x_1+x_2+...+x_n$ são os n valores de um conjunto de dados, a média aritmética$(\bar{x})$ é determinada da seguinte maneira:

$\bar{x}=\frac{x_1+x_2+...+x_n}{n}$

A partir de agora vamos usar as bibliotecas Python para resolver nossos problemas. Mas se você tiver a curiosidade de saber como o problema poderia ser resolvido usando somente os conceitos não deixe de olhar o material em Anexo. 

**Exemplo 1**: Determine a idade média dos passageiros do Titanic:

O *Exemplo 1* nos mostrou que a idade média das pessoas que estavam no Titanic foi de aproximadamente 29 anos, mas será que esse número representa bem todos os passageiros da embarcação? 

Para tentar responder essa pergunta vamos dividir o nosso conjunto de dados em subconjuntos menores, buscaremos características que separam o conjunto em subconjunto complementares, por exemplo: Survived, Pclass, Sex, Age, Siblings/Spouses Aboard.

Essa ideia de dividir o conjunto original em subconjuntos com caracteríscas determinadas é conhecida como <u>Estratificação</u>.

**Exemplo 2:** Encontre a idade média das pessoas do navio de acordo com o sexo.

Vimos na aula passada que é possível agrupar atributos e retirar as médias usando a função groupby.

**Obs:** Existem outras médias, porém cada uma delas é usada em situações específicas. A saber:

- **Média Ponderada:** Você deve usar uma média ponderada quando deseja atribuir mais importância a alguns números em um conjunto de dados do que a outros. Isso é útil em cenários onde um evento pode ter vários resultados positivos ou negativos, e a magnitude desses resultados varia.

- **Média Harmônica:** A média harmônica é calculada como o número de valores dividido pela soma do inverso de cada valor. É apropriada quando os dados representam grandezas que são inversamente proporcionais, como taxas.

- **Média Geométrica:** A média geométrica é calculada como a raiz N-ésima do produto de todos os valores, onde N é o número de valores. É útil quando os dados estão em uma escala multiplicativa, como em situações envolvendo crescimento ou taxa de variação entre diferentes unidades de medida.

### Mediana

A **mediana** da amostra é o elemento central de um conjunto de dados ordenado (crescente ou decrescente). Se o número de elementos 𝑛 do conjunto de dados for ímpar, então a mediana é o valor na posição do meio: $\frac{(𝑛 + 1)}{2}$. Se 𝑛 for par, então a mediana é a média aritmética dos dois valores no meio, ou seja, os itens nas posições $\frac{𝑛}{2}$ e $\frac{𝑛 + 1}{2}$.

**Exemplo 3:** Se tivermos os pontos de dados 3, 5, 1, 2 e 8, o valor mediano será 3, pois 3 é o elemento central após a ordenação do conjunto (1, 2, 3, 5, 8). Se os pontos de dados forem 3, 5, 1 e 8, então a mediana será 4, que é a média dos dois elementos centrais da sequência ordenada (1, 3, 5, 8).

**Importante:** A principal diferença entre o comportamento da média e da mediana está relacionada aos valores extremos (outliers) do conjunto de dados. De uma maneira geral:

- Se você colocar um valor discrepante em um conjunto de dados, a média aumentará, mas o valor da mediana permanecerá inalterado.
- Se você remover um valor discrepante de um conjunto de dados, a média diminuirá, mas a mediana continuará a mesma.

Você pode usar a comparação entre a média e a mediana como uma maneira de detectar discrepâncias e assimetrias em seus dados.

**Exemplo 4:** Vamos determinar a mediana das Idades:

Assim:

### Moda

A <u>moda</u> da amostra é o valor no conjunto de dados que ocorre com mais frequência. Se não houver um único valor desse tipo, o conjunto será multimodal, pois possui vários valores modais. 

**Exemplo 5:** Determine a moda dos conjuntos a seguir:

Aqui precisamos usar uma lambda function para encontrar a moda da idade por Sexo e Classe.

## Medidas de Localização

O **percentil** 𝑝 da amostra é o elemento no conjunto de dados tal que 𝑝% dos elementos no conjunto de dados são menores ou iguais a esse valor. Além disso, (100 − 𝑝)% dos elementos são maiores ou iguais a esse valor. Se houver dois desses elementos no conjunto de dados, o percentil 𝑝 da amostra é a média aritmética deles.

Por exemplo, os percentis que dividem o conjunto de dados em quatro partes iguais são chamados de Quartis. Cada conjunto de dados tem três quartis, que são:

- O primeiro quartil $(Q_1)$ é o percentil 25 da amostra. Ele divide aproximadamente 25% dos menores itens do restante do conjunto de dados.
- O segundo quartil $(Q_2)$ é o percentil 50 da amostra, também conhecido como a mediana. Aproximadamente 25% dos itens situam-se entre o primeiro e o segundo quartis, e outros 25% entre o segundo e o terceiro quartis.
- O terceiro quartil $(Q_3)$ é o percentil 75 da amostra. Ele divide aproximadamente 25% dos maiores itens do restante do conjunto de dados.

**Exemplo 6:** Determine os quartis das idades dos passageiros do Titanic.